In [1]:
!pip install kumaraswamy

In [2]:
from kumaraswamy import kumaraswamy

## Experiments with Kumaraswamy Function

In [3]:
d1 = kumaraswamy(a=0.1, b=0.7)

In [4]:
d1.pdf(0.9)

array(0.30210796)

In [5]:
d1.cdf(0.21)

array(0.7418337)

In [6]:
d1.var

0.07907948744725794

In [7]:
d1.mean

0.17121255852442666

### CCTE

In [11]:
from scipy.special import hyp2f1
from scipy.special import betainc
#The incomplte Beta function


In [12]:
def num_1(alpha,t,a,b,theta):
    v1=-(1/a)
    v2=b
    v3=1+b
    v4=(1-alpha)**(1/b)
##This is the first term in the numerator
    p1=(t-1)*(theta*t-1)
    p2=(1-alpha)*(hyp2f1(v1,v2,v3,v4))
    return p1*p2 #The product of 2  terms is the first term
    

In [55]:
def num_2(alpha,t,a,b,theta):
    #This gives the negative of the second term
    a3=1+(1/a)
    p=alpha**b
    b3=2*b
    p3=2*(theta)*t*(t-1)*b
    p4=[betainc(a3,b,1)+betainc(a3,b3,1)-betainc(a3,b,p)-betainc(a3,b3,p)]
    prod=p4[0]*p3
    return prod


In [56]:
def FGM_copula(alpha,t,a,b,theta,a1,b1):
    d1 = kumaraswamy(a, b)
    d2 = kumaraswamy(a1, b1)
    s1 = d1.cdf(alpha)
    s2 = d2.cdf(t)
    copula=(s1)*(s2)+theta*(s1)*(s2)*(1-s1)*(1-s2)
    return copula
    
    
    
def deno(alpha,t,a,b,theta,a1,b1):
    t1=1-alpha-t
    t2= FGM_copula(alpha,t,a,b,theta,a1,b1)
    return t1+t2
    

In [57]:
FGM_copula(0.3,0.5,0.7,0.8,2,0.6,0.8)

0.32232193347566945

In [66]:
def ccte(alpha,a,b,t,a1,b1,theta):
    num=num_1(alpha,t,a,b,theta)+num_2(alpha,t,a,b,theta)
    den=deno(alpha,t,a,b,theta,a1,b1)
    return num/den
    

In [69]:
ccte(0.2,0.5,0.8,0.9,0.8,0.9,0.4)

-0.3503350944639705

In [60]:
num_2(0.2,0.7,0.8,0.9,4)

-2.838277127208187

In [74]:
import numpy as np
def matrix(n):
    a=(n,n)
    return np.zeros(a)

In [76]:
c=matrix(3)

In [77]:
c[1,2]=25

In [78]:
c

array([[ 0.,  0.,  0.],
       [ 0.,  0., 25.],
       [ 0.,  0.,  0.]])

In [84]:
def update_matrix(mat,i,j,alpha,a,b,t,a1,b1,theta):
    #input here whcih element we want to update(exact index)
    mat[i-1,j-1]=ccte(alpha,a,b,t,a1,b1,theta)
    

In [85]:
update_matrix(c,2,2,0.2,0.5,0.8,0.9,0.8,0.9,0.4)

In [86]:
c

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35033509, 25.        ],
       [ 0.        ,  0.        , -0.35033509]])